In [56]:
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [57]:
# open the shapefile
# gdf = gpd.read_file('/app/notebooks/rapprochements/Bordeaux/20240625 - ECHANTILLON BUILDING RNB/063_3_063_8_162_Building.shp')
gdf = gpd.read_file('/app/notebooks/rapprochements/Bordeaux/Bordeaux-tout/Bordeaux.shp')
gdf = gdf[['BUILDINGID', 'geometry']]
# print current crs
print(gdf.crs)

gdf = gdf.to_crs(epsg=4326)
gdf['geometry'] = gdf['geometry'].force_2d()
print(gdf.head())

/venv/lib/python3.12/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(


EPSG:3945
  BUILDINGID                                           geometry
0   28109912  POLYGON ((-0.73855 44.84228, -0.73853 44.84225...
1   28113973  POLYGON ((-0.65259 44.83922, -0.6526 44.83922,...
2   28118468  POLYGON ((-0.69782 44.83392, -0.69782 44.83392...
3   28100527  POLYGON ((-0.65619 44.84985, -0.6562 44.84984,...
4   28114929  POLYGON ((-0.66237 44.84621, -0.66238 44.84617...


In [58]:
# number of buildings
print(len(gdf))

274686


In [59]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection
from concurrent.futures import ThreadPoolExecutor
import numpy as np

# sample = gdf.sample(1000)

# split the dataframe in 50 smaller dataframes
dfs = np.array_split(gdf, 50)



# apply a function to each row, return a dataframe
def bordeaux_rnb_intersection(row):
    cursor = connection.cursor()
    geometry = row['geometry']
    sql = f"""with areas as (
select
	rnb_id,
	ST_AREA(ST_INTERSECTION(shape,
	ST_GeomFromText('{geometry}', 4326))) as area_intersection,
	ST_AREA(shape) as area_rnb,
	ST_AREA(ST_GeomFromText('{geometry}', 4326)) as area_bordeaux
from
	batid_building
where
	ST_Intersects(shape, ST_GeomFromText('{geometry}', 4326)) and is_active=true)
        select
	{row['BUILDINGID']} as BUILDINGID, rnb_id, area_intersection / NULLIF(area_rnb, 0) as rnb_inclu_bdx, area_intersection / NULLIF(area_bordeaux, 0) as bdx_inclu_rnb
from
	areas"""

    cursor.execute(sql)

    results = cursor.fetchall()

    df = None
    if len(results) == 0:
        result = [{'BUILDINGID': row['BUILDINGID'], 'rnb_id': None, 'rnb_inclu_bdx': None, 'bdx_inclu_rnb': None}]
        df = pd.DataFrame(result)
    else:
        df = pd.DataFrame(results, columns=['BUILDINGID', 'rnb_id', 'rnb_inclu_bdx', 'bdx_inclu_rnb'])
    return df

def execute(df):
    df_copy = df.copy()
    df_copy = df_copy.apply(bordeaux_rnb_intersection, axis=1)
    df_copy = pd.concat(df_copy.values)
    return df_copy


with ThreadPoolExecutor(max_workers=50) as executor:
    results = executor.map(execute, dfs)
    gdf_compute = pd.concat(results)
    gdf_compute.to_csv('/app/notebooks/rapprochements/Bordeaux/intermediate_results_all.csv', index=False)



/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.p

In [62]:
gdf_compute.head()

,BUILDINGID,rnb_id,rnb_inclu_bdx,bdx_inclu_rnb
0,28109912,Y2PJP3T6D9SJ,1.000000,0.347379
1,28113973,PTXPN1FGPC1Y,0.999957,0.330095
2,28113973,GPYGZFHRAHS5,1.000000,0.094151
3,28113973,68K6H89MSEW8,0.995663,0.518983
4,28118468,NaN,NaN,NaN


In [63]:
# gdf_compute = pd.read_csv('/app/notebooks/rapprochements/Bordeaux/intermediate_results.csv')
gdf_compute = pd.read_csv('/app/notebooks/rapprochements/Bordeaux/intermediate_results_all.csv')
gdf_compute.head()

,BUILDINGID,rnb_id,rnb_inclu_bdx,bdx_inclu_rnb
0,28109912,Y2PJP3T6D9SJ,1.000000,0.347379
1,28113973,PTXPN1FGPC1Y,0.999957,0.330095
2,28113973,GPYGZFHRAHS5,1.000000,0.094151
3,28113973,68K6H89MSEW8,0.995663,0.518983
4,28118468,NaN,NaN,NaN


In [64]:
gdf_compute_2 = gdf_compute.copy()

# a match is when both intersection ratios are above 0.8
gdf_compute_2['match_1_1'] = (gdf_compute_2['rnb_inclu_bdx'] > 0.8) & (gdf_compute_2['bdx_inclu_rnb'] > 0.7)
gdf_compute_2['rnb_inclu_bdx_b'] = (gdf_compute_2['rnb_inclu_bdx'] > 0.8) & (gdf_compute_2['bdx_inclu_rnb'] <= 0.7)
gdf_compute_2['bdx_inclu_rnb_b'] = (gdf_compute_2['bdx_inclu_rnb'] > 0.7) & (gdf_compute_2['rnb_inclu_bdx'] <= 0.8)
gdf_compute_2['small_intersection'] = (gdf_compute_2['rnb_inclu_bdx'] <= 0.8) & (gdf_compute_2['bdx_inclu_rnb'] <= 0.7)

# print(gdf_compute_2.head(10))

# print gdf_compute_2 for the ID 6318700
# print(gdf_compute_2[gdf_compute_2['BUILDINGID'] == 6318700])

intersections = gdf_compute_2[gdf_compute_2['rnb_id'].notnull()].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
match_1_1 = gdf_compute_2[gdf_compute_2['match_1_1']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
rnb_inclu_bdx = gdf_compute_2[gdf_compute_2['rnb_inclu_bdx_b']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
bdx_inclu_rnb = gdf_compute_2[gdf_compute_2['bdx_inclu_rnb_b']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
small_intersection = gdf_compute_2[gdf_compute_2['small_intersection']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()

# merge all the dataframes
df_merge = pd.DataFrame(gdf_compute['BUILDINGID'].unique(), columns=['BUILDINGID'])
df_merge = df_merge.merge(intersections, on='BUILDINGID', how='left', suffixes=('', '_intersections'))
df_merge = df_merge.merge(match_1_1, on='BUILDINGID', how='left', suffixes=('', '_match_1_1'))
df_merge = df_merge.merge(rnb_inclu_bdx, on='BUILDINGID', how='left', suffixes=('', '_rnb_inclu_bdx'))
df_merge = df_merge.merge(bdx_inclu_rnb, on='BUILDINGID', how='left', suffixes=('', '_bdx_inclu_rnb'))
df_merge = df_merge.merge(small_intersection, on='BUILDINGID', how='left', suffixes=('', '_small_intersection'))


# # clean None values in lists
# df_merge['rnb_id'] = df_merge['rnb_id'].apply(lambda l: [rnb_id for rnb_id in l if rnb_id is not None])

df_merge = df_merge.fillna("").apply(list)


df_merge.head(20)


,BUILDINGID,rnb_id,rnb_id_match_1_1,rnb_id_rnb_inclu_bdx,rnb_id_bdx_inclu_rnb,rnb_id_small_intersection
0,28109912,[Y2PJP3T6D9SJ],,[Y2PJP3T6D9SJ],,
1,28113973,"[PTXPN1FGPC1Y, GPYGZFHRAHS5, 68K6H89MSEW8]",,"[PTXPN1FGPC1Y, GPYGZFHRAHS5, 68K6H89MSEW8]",,
2,28118468,,,,,
3,28100527,"[MYSHX1RJT8EJ, VG6AVFQF9FSA, 544P3RY978C8]",,"[MYSHX1RJT8EJ, VG6AVFQF9FSA, 544P3RY978C8]",,
4,28114929,"[CNSQT2AMVFNG, 5FTWFC56S579]",[5FTWFC56S579],,,[CNSQT2AMVFNG]
5,28100000,"[WFPTM13MPPM8, QVKGY9NVTD7N]",,"[WFPTM13MPPM8, QVKGY9NVTD7N]",,
6,28100001,[QDME34HYNJG8],[QDME34HYNJG8],,,
7,28100002,"[8W274KYCQXVX, SPFEBGPVEXMR]",,"[8W274KYCQXVX, SPFEBGPVEXMR]",,
8,28100005,[JNM9R6V33889],,[JNM9R6V33889],,
9,28100006,,,,,


In [65]:
# print number of rows
N = len(df_merge)
print("Nombre de bâtiments dans l'échantillon")
print(N)

print("Nombre de bâtiments sans intersection")
n = len(df_merge[df_merge['rnb_id'].apply(lambda l: len(l) == 0)])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments avec intersection")
n = len(df_merge[df_merge['rnb_id'].apply(lambda l: len(l) > 0)])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments avec match 1:1")
n = len(df_merge[df_merge['rnb_id_match_1_1'].apply(lambda l: len(l) == 1)])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments avec match 1:1 ou un seul batiment RNB inclu dans Bordeaux")
n = len(df_merge[df_merge['rnb_id_match_1_1'].apply(lambda l: len(l) == 1) | (df_merge['rnb_id_rnb_inclu_bdx'].apply(lambda l: len(l) == 1))])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments qui contiennent plusieurs bâtiments RNB inclus")
n = len(df_merge[df_merge['rnb_id_rnb_inclu_bdx'].apply(lambda l: len(l) > 1) & df_merge['rnb_id_match_1_1'].apply(lambda l: len(l) == 0)])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments qui sont inclus dans un bâtiment RNB")
n = len(df_merge[df_merge['rnb_id_bdx_inclu_rnb'].apply(lambda l: len(l) > 0) & df_merge['rnb_id_match_1_1'].apply(lambda l: len(l) == 0)])
print(n)
print(f"soit {round(n/N*100, 1)}%")

print("Nombre de bâtiments avec uniquement des petites intersections")
n = len(df_merge[df_merge['rnb_id_small_intersection'].apply(lambda l: len(l) > 0) & df_merge['rnb_id_match_1_1'].apply(lambda l: len(l) == 0) & df_merge['rnb_id_rnb_inclu_bdx'].apply(lambda l: len(l) == 0) & df_merge['rnb_id_bdx_inclu_rnb'].apply(lambda l: len(l) == 0)])
print(n)
print(f"soit {round(n/N*100, 1)}%")



Nombre de bâtiments dans l'échantillon
274667
Nombre de bâtiments sans intersection
30442
soit 11.1%
Nombre de bâtiments avec intersection
244225
soit 88.9%
Nombre de bâtiments avec match 1:1
141376
soit 51.5%
Nombre de bâtiments avec match 1:1 ou un seul batiment RNB inclu dans Bordeaux
183504
soit 66.8%
Nombre de bâtiments qui contiennent plusieurs bâtiments RNB inclus
29539
soit 10.8%
Nombre de bâtiments qui sont inclus dans un bâtiment RNB
12824
soit 4.7%
Nombre de bâtiments avec uniquement des petites intersections
18248
soit 6.6%


In [66]:
# df_merge.to_csv('/app/notebooks/rapprochements/Bordeaux/results_echantillon.csv', index=False)
df_merge['BUILDINGID'] = df_merge['BUILDINGID'].astype(str)
df_merge['BUILDINGID'] = df_merge['BUILDINGID'].str.zfill(8)
df_merge.to_csv('/app/notebooks/rapprochements/Bordeaux/results_all.csv', index=False)


In [67]:
# filter df with len of rnb_id_rnb_inclu_bdx > 1
df_merge_filtered = df_merge[['BUILDINGID', 'rnb_id_rnb_inclu_bdx', 'rnb_id_match_1_1']]
df_merge_filtered = df_merge_filtered[df_merge_filtered['rnb_id_rnb_inclu_bdx'].apply(lambda l: len(l) > 1) & df_merge_filtered['rnb_id_match_1_1'].apply(lambda l: len(l) == 0)]
building_ids_filtered = df_merge_filtered['BUILDINGID']
print(len(building_ids_filtered))

29539


In [68]:
building_ids_str = building_ids_filtered.astype(str)
# pad with 0 to have 8 digits
building_ids_str = building_ids_str.str.zfill(8)

gdf_filtered = gdf[gdf['BUILDINGID'].isin(building_ids_str)]
print(gdf_filtered.head())
# write to file
gdf_filtered.to_file('/app/notebooks/rapprochements/Bordeaux/buildings_sous_decoupes.shp')

   BUILDINGID                                           geometry
1    28113973  POLYGON ((-0.65259 44.83922, -0.6526 44.83922,...
3    28100527  POLYGON ((-0.65619 44.84985, -0.6562 44.84984,...
5    28100000  POLYGON ((-0.68489 44.83838, -0.6849 44.83838,...
7    28100002  POLYGON ((-0.65173 44.83282, -0.65173 44.83282...
13   28100010  POLYGON ((-0.6265 44.81846, -0.62654 44.81845,...


In [54]:
print(len(gdf_filtered))

29553
